# Acute Coronary Syndrome Study

Lets see if we can run [the queries](http://build.fhir.org/ig/HL7/vulcan-rwd/acs.html) for cohort criteria.

See:
- https://clinicaltrials.gov/ct2/show/NCT02190123
- https://confluence.hl7.org/display/FHIR/Public+Test+Servers

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pete88b/vulcan_rwd_ig/blob/main/rwd_ig_cohort_building.ipynb)

## Please note

This notebook is a work in progress

For the rough notes we worked on during the connectathon, please see
https://github.com/pete88b/vulcan_rwd_ig/blob/main/getting_started.ipynb

In [ ]:
import requests, json, collections

In [ ]:
class FhirPathDict(collections.UserDict):
    "Wraps a `dict` to allow dot search of nested values"
    def __getitem__(self, key):
        "Allows dot search via subscription"
        for _attr_path in key.split(' OR '):
            r, found, _attrs = self.data, True, []
            for _attr in _attr_path.split('.'):
                _attrs.append(_attr)
                if not isinstance(r, dict):
                    raise Exception(f'''Expected "{'.'.join(_attrs)}" to be a `dict` but found {r}''')
                if not _attr in r:
                    found = False
                    break
                r = r[_attr]
                if isinstance(r, list) and r:
                    r = r[0] # TODO: is it OK to just pull the 1st item from the list?
            if found:
                return FhirPathDict(r) if isinstance(r, dict) else r

In [ ]:
d = FhirPathDict({'resource': {'resourceType': 'TestResource', 'Other-Key': 'Other-Value'}})
d['resource.resourceType'], d['resource.id']

('TestResource', None)

In [ ]:
class FhirClient:
    "Helps to GET FHIR resources"
    def __init__(self, api_base, x_api_key=None):
        self.api_base = api_base
        self.request_headers = {}
        if x_api_key is not None:
            self.request_headers['x-api-key'] = x_api_key
        self.default_params = {}
    
    def get_as_response(self, resource_type, params=None):
        "GET FHIR resources of `resource_type` and return python `reponse`"
        url = f'{self.api_base}/{resource_type}'
        params = self.default_params if params is None else params
        response = requests.get(url, params, headers=self.request_headers)
        print('GET', response.url, response.status_code)
        return response
    
    def get_as_raw_json(self, resource_type, id_or_params=None):
        "GET FHIR resources of `resource_type` in JSON format"
        return self.get_as_response(resource_type, id_or_params).json()
    
    def get_next_as_raw_json(self, json_response):
        "GET the next set of results"
        for link in json_response['link']:
            if link['relation'] == 'next':
                url = link['url']
                print('GET', url)
                return requests.get(url, headers=self.request_headers).json()

    def get_all_entries_as_raw_json(self, resource_type, params=None, page_limit=100):
        "Return a list of entries of `resource_type` in JSON format"
        page_count, result = 0, []
        bundle = self.get_as_raw_json(resource_type, params)
        total = bundle.get('total', 'Unknown')
        while bundle is not None:
            if 'entry' not in bundle:
                raise Exception(f'Expected a bundle but found', bundle) # might be {'resourceType': 'OperationOutcome' ... 
            result.extend(bundle['entry']) # todo check for OperationOutcome etc in `entry`
            page_count += 1
            if page_count > page_limit:
                print('Stopping early. Will return', len(result), 'entries out of total', total)
                break
            bundle = client.get_next_as_raw_json(bundle)
        def _expected_resource_type(resource):
            _type = resource.get('resource', {}).get('resourceType', None)
            if _type != resource_type:
                print('Removing resource. Expected', resource_type, 'but found', _type)
                return False
            return True
        result = [r for r in result if _expected_resource_type(r)]
        result = [FhirPathDict(r) for r in result]
        return result
    
    def get_all_resources_as_raw_json(self, resource_type, params=None, page_limit=100):
        "Return a list of resources of `resource_type` in JSON format"
        result = self.get_all_entries_as_raw_json(resource_type, params, page_limit)
        result = [r['resource'] for r in result]
        return result
    
    def get_by_reference(self, reference):
        "Return a resource read from a FHIR server by reference, as a list containg a single bundle entry"
        if reference.startswith(self.api_base):
            reference = reference[len(self.api_base):].strip('/')
        if reference.startswith('http'):
            print(f'WARNING: Found reference {reference} that does not start with {api_base}')
            return []
        resource_type, id = reference.split('/')
        single_resource = self.get_as_raw_json(resource_type, id)
        return [dict(fullUrl = f'{self.api_base}/{resource_type}/{id}', resource = single_resource)]

In [ ]:
client = FhirClient('https://ips.health/fhir')
# client = FhirClient('https://fhir.rykpjsvemdtg.workload-prod-fhiraas.isccloud.io')
# patient_entries = client.get_all_entries_as_raw_json('Patient')
patient_entries = client.get_all_resources_as_raw_json('Patient', page_limit=2)
# patient_entries = client.get_all_entries_as_raw_json('Patient', {'birthdate': 'le2002-09-01', 'gender:missing': 'false'})
# patient_entries

GET https://ips.health/fhir/Patient 200
GET https://ips.health/fhir?_getpages=4a845c81-78ff-4326-81bf-acdded1811c2&_getpagesoffset=20&_count=20&_pretty=true&_bundletype=searchset
GET https://ips.health/fhir?_getpages=4a845c81-78ff-4326-81bf-acdded1811c2&_getpagesoffset=40&_count=20&_pretty=true&_bundletype=searchset
Stopping early. Will return 60 entries out of total 187


In [ ]:
patient_entries[0].data

{'resourceType': 'Patient',
 'id': '1',
 'meta': {'versionId': '1',
  'lastUpdated': '2022-09-06T21:12:47.040+00:00',
  'source': '#fUPtWa97RV5lOt2l'},
 'text': {'status': 'generated',
  'div': '<div xmlns="http://www.w3.org/1999/xhtml"><p><b>Generated Narrative</b></p><div style="display: inline-block; background-color: #d9e0e7; padding: 6px; margin: 4px; border: 1px solid #8da1b4; border-radius: 5px; line-height: 60%"><p style="margin-bottom: 0px">Resource &quot;2b90dd2b-2dab-4c75-9bb9-a355e07401e8&quot; </p></div><p><b>identifier</b>: id: 574687583</p><p><b>active</b>: true</p><p><b>name</b>: Martha DeLarosa </p><p><b>telecom</b>: <a href="tel:+31788700800">+31788700800</a></p><p><b>gender</b>: female</p><p><b>birthDate</b>: 1972-05-01</p><p><b>address</b>: Laan Van Europa 1600 Dordrecht 3317 DB NL </p><h3>Contacts</h3><table class="grid"><tr><td>-</td><td><b>Relationship</b></td><td><b>Name</b></td><td><b>Telecom</b></td><td><b>Address</b></td></tr><tr><td>*</td><td>mother <span st

show &darr; that it's possible to dot search
- on the results of `get_all_resources_as_raw_json` e.g. `'contact.relationship.coding.system'` and
- on the results returned by dot search

TODO: xxx explain this better

In [ ]:
_r = patient_entries[0]['contact.relationship']
patient_entries[0]['contact.relationship.coding.system'], _r['coding.code']

('http://terminology.hl7.org/CodeSystem/v2-0131', 'C')

In [ ]:
client = FhirClient('https://ips.health/fhir')
r = client.get_as_raw_json('Patient')

GET https://ips.health/fhir/Patient 200


In [ ]:
len(r['entry'])

20

In [ ]:
len(client.get_next_as_raw_json(r)['entry'])

GET https://ips.health/fhir?_getpages=801ef8f5-ea46-4b41-a775-acb010c4f4ba&_getpagesoffset=20&_count=20&_pretty=true&_bundletype=searchset


20

In [ ]:
len(patient_entries)

187

In [ ]:
patient_entries

[{'fullUrl': 'https://ips.health/fhir/Patient/1',
  'resource': {'resourceType': 'Patient',
   'id': '1',
   'meta': {'versionId': '1',
    'lastUpdated': '2022-09-06T21:12:47.040+00:00',
    'source': '#fUPtWa97RV5lOt2l'},
   'text': {'status': 'generated',
    'div': '<div xmlns="http://www.w3.org/1999/xhtml"><p><b>Generated Narrative</b></p><div style="display: inline-block; background-color: #d9e0e7; padding: 6px; margin: 4px; border: 1px solid #8da1b4; border-radius: 5px; line-height: 60%"><p style="margin-bottom: 0px">Resource &quot;2b90dd2b-2dab-4c75-9bb9-a355e07401e8&quot; </p></div><p><b>identifier</b>: id: 574687583</p><p><b>active</b>: true</p><p><b>name</b>: Martha DeLarosa </p><p><b>telecom</b>: <a href="tel:+31788700800">+31788700800</a></p><p><b>gender</b>: female</p><p><b>birthDate</b>: 1972-05-01</p><p><b>address</b>: Laan Van Europa 1600 Dordrecht 3317 DB NL </p><h3>Contacts</h3><table class="grid"><tr><td>-</td><td><b>Relationship</b></td><td><b>Name</b></td><td><b>T